In [85]:
#load packages
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

#import qLearn.ipynb
import import_ipynb
from qLearn import *

#load the word2vec model
model = Word2Vec.load('codenames2.model')

In [86]:
from itertools import combinations

def QLClueMaster(board, bluelist, redlist, assassin, model):
    #create the board master list with wordvectors and penalties
    board_max = []
    for word in board:
        if word in bluelist:
            penalty = -1
        elif word in redlist:
            penalty = -10
        elif word == assassin:
            penalty = -100
        else:
            penalty = -5
    board_max.append([word, model.wv[word], penalty])
    #set combo list and scores list to empty
    combos = []
    scores = []
    #loop from combos of 1 to the length of your wordlist
    if len(bluelist) == 1:
        p1 = 0
        p2 = 0
    elif len(bluelist) < 4:
        p1 = 1
        p2=0
    elif len(bluelist) <= 5:
        p1 = 2
        p2 = 1
    else:
        p1 = 3
        p2 = 2
    for i in range(p1,len(bluelist)-p2):
        #get all possible combos for i words in the bluelist
        combos = list(combinations(bluelist, i))
        #for each combo, get the average vector and find the most similar word to that vector
        for combo in combos:
            blue_avg = np.zeros(100)
            #get the average vector for the combo
            for word in combo:
                blue_avg += model.wv[word]
            
            blue_avg = blue_avg/len(combo)
            #get the top i + 1 most similar words to the average vector (clues)
            clues = model.wv.similar_by_vector(blue_avg, topn=(i+1))
            #try the first clue, if it's in the combo, try the next one, and so on until you find a clue that isn't in the combo
            clue = clues[0][0]
            j = 1
            while clue in combo:
                clue = clues[j][0]
                j += 1
            clue = clues[j-1][0]
            #get the score for the clue
            score = 0
            for word in board_max:
                score += model.wv.similarity(clue, word[0]) * word[2]
            #adjust the score by the length of the combo
            #score = score / len(combo)
            #add the clue, score, and combo to the scores list
            scores.append([clue, score, combo])

    #sort the scores list by the score
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
   
    #return the length of word list, word list, and clue
    return scores[0]
    


In [87]:
def guess_word(board, model, clue, n):
    #given a list of potential words, return the n most similar words in the list to the clue from the board
    #board is a list of words
    #model is a word2vec model
    #clue is a string
    #n is an integer
    #return a list of n words
    #if n is greater than the length of the board, return the board
    if n > len(board):
        return board 
    #get the vector representation of the clue
    clue_vector = model.wv[clue]
    #get the vector representation of the board
    board_vectors = [model.wv[word] for word in board]
    #get the similarity between the clue and each word in the board
    similarities = [np.dot(clue_vector, board_vector)/(np.linalg.norm(clue_vector)*np.linalg.norm(board_vector)) for board_vector in board_vectors]
    #get the indices of the n most similar words
    indices = np.argsort(similarities)[::-1][:n]
    #return the n most similar words
    return [board[i] for i in indices]

In [88]:
#get possible wordlist from the model vocab
wordlist = list(model.wv.key_to_index.keys())
# Generate a random list of 25 unique words
board = random.sample(wordlist, 25)

# Split the words into bluelist, redlist, and assassin
bluelist = board[:9]
redlist = board[9:17]
assassin = board[24:]
#print the board and the like
print("Board:", board)
print("Blue Team:", bluelist)
print("Red Team:", redlist)
print("Assassin:", assassin)
print(' ')

#run the QLClueMaster function
clue = QLClueMaster(board, bluelist, redlist, assassin, model)

#run the guess_word function
guess_word(board, model, clue[0], len(clue[2]))

Board: ['board', 'match', 'thumb', 'buck', 'theater', 'torch', 'brush', 'hand', 'pilot', 'track', 'revolution', 'genius', 'crown', 'spider', 'mole', 'stadium', 'chick', 'train', 'litter', 'bill', 'chocolate', 'ghost', 'post', 'tube', 'time']
Blue Team: ['board', 'match', 'thumb', 'buck', 'theater', 'torch', 'brush', 'hand', 'pilot']
Red Team: ['track', 'revolution', 'genius', 'crown', 'spider', 'mole', 'stadium', 'chick']
Assassin: ['time']
 


['thumb', 'match', 'theater']

In [89]:
def test_model(model, datadict):
    wordlist = list(model.wv.key_to_index.keys())
    # Generate a random list of 25 unique words
    board = random.sample(wordlist, 25)

    # Split the words into bluelist, redlist, and assassin
    bluelist = board[:9]
    redlist = board[9:17]
    assassin = board[24:]

    correct = 0
    blue_points = 0
    red_points = 0
    neutral_points = 0
    assassin_bool = False
    while(blue_points < 9 and red_points < 8 and assassin_bool == False):
        clue =QLClueMaster(board, bluelist, redlist, assassin, model)
        predicted = guess_word(board, model, clue[0], len(clue[2]))
        actual = clue[2]
        for word in predicted:
            
            board.remove(word)
            if word in actual:
                correct += 1
            if word in bluelist:
                blue_points += 1
                bluelist.remove(word)
            if word in redlist:
                red_points += 1
                redlist.remove(word)
            if word == assassin:
                assassin_bool = True
            else:
                neutral_points += 1
            #remove the word from the board

            #pick two random words from the redlist and add them to a list
        if len(redlist) > 1:
            #generate random number between 0 and half the length of the redlist
            rand1 = random.randint(0, len(redlist)//2)
            #pick rand1 num words from the redlist
            red_words = random.sample(redlist, rand1)
            #remove the words from the redlist
            for word in red_words:
                redlist.remove(word)
                board.remove(word)
                red_points += 1

        elif len(redlist) == 1:
            red_words = [redlist[0]]
            redlist.remove(red_words[0])
            board.remove(red_words[0])
            
            if len(redlist) == 0:
                break
            if len(bluelist) == 0:
                break

   
              
    if assassin_bool:
        datadict['correct'] += correct
        datadict['blue_points'] += blue_points
        datadict['red_points'] += red_points
        datadict['neutral_points'] += neutral_points
        datadict['assassin'] += 1
    if len(bluelist) == 0:
        datadict['correct'] += correct
        datadict['blue_points'] += blue_points
        datadict['red_points'] += red_points
        datadict['neutral_points'] += neutral_points
        datadict['wins'] += 1
    if len(redlist) == 0:
        datadict['correct'] += correct
        datadict['blue_points'] += blue_points
        datadict['red_points'] += red_points
        datadict['neutral_points'] += neutral_points
    return;

In [92]:
def simulator(model):
    datadict = {}   
    datadict['wins'] = 0
    datadict['correct'] = 0
    datadict['blue_points'] = 0
    datadict['red_points'] = 0
    datadict['neutral_points'] = 0
    datadict['assassin'] = 0
    for i in range(1000):
        test_model(model, datadict)
    print(datadict)
    return;

In [93]:
#run the test_model function
simulator(model)

/var/folders/_8/5fm0bg890md4d79y3w9f4vvm0000gn/T/ipykernel_68112/2377024429.py:42: RuntimeWarning: invalid value encountered in divide
  blue_avg = blue_avg/len(combo)
